<a href="https://colab.research.google.com/github/SLCFLAB/Fintech2022/blob/main/ML_day13_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Word Embedding

자연어차리는 언어데이터를 기반으로 정보를 추출하고 분석, 활용하는 기술
- 분석 단위 설정: 토큰화(Tokenize)
- 불필요한 단어 제거: 스토핑(Stopping)
> a, the, 은, 는, 이, 가
- 단어의 형태 고려: 스테밍(Stemming)
> ed, ing, 하다, 했다, 할 것이다


이렇게 전처리 한 후 어떻게 해야할까?
> Word Embedding(단어를 벡터화 하는 것)
- Bag of Words
- Word2Vec

In [1]:
import torch
import torch.nn as nn

In [2]:
train_data = '서울대 핀테크 과정 딥러닝 수업 시간입니다'

In [3]:
word_set = set(train_data.split())

In [4]:
vocab = {word: i+2 for i, word in enumerate(word_set)}
vocab['<unk>'] = 0
vocab['<pad>'] = 1
print(vocab)

{'서울대': 2, '수업': 3, '딥러닝': 4, '시간입니다': 5, '과정': 6, '핀테크': 7, '<unk>': 0, '<pad>': 1}


In [5]:
len(vocab)

8

In [6]:
embedding_table = torch.FloatTensor([
                               [ 0.0,  0.0,  0.0],
                               [ 0.0,  0.0,  0.0],
                               [ 0.2,  0.9,  0.3],
                               [ 0.1,  0.5,  0.7],
                               [ 0.2,  0.1,  0.8],
                               [ 0.4,  0.1,  0.1],
                               [ 0.1,  0.8,  0.9],
                               [ 0.6,  0.1,  0.1]])

In [7]:
sample = '핀테크 과정 수업 재밌습니다'.split()
idxes = []

# 각 단어를 정수로 변환
for word in sample:
  try:
    idxes.append(vocab[word])
  # 단어 집합에 없는 단어일 경우 <unk>로 대체된다.
  except KeyError:
    idxes.append(vocab['<unk>'])
idxes = torch.LongTensor(idxes)

# 각 정수를 인덱스로 임베딩 테이블에서 값을 가져온다.
lookup_result = embedding_table[idxes, :]
print(lookup_result)

tensor([[0.6000, 0.1000, 0.1000],
        [0.1000, 0.8000, 0.9000],
        [0.1000, 0.5000, 0.7000],
        [0.0000, 0.0000, 0.0000]])


nn.Embedding 사용

In [8]:
train_data = '서울대 핀테크 과정 딥러닝 수업 시간입니다'

# 중복을 제거한 단어들의 집합인 단어 집합 생성.
word_set = set(train_data.split())

# 단어 집합의 각 단어에 고유한 정수 맵핑.
vocab = {tkn: i+2 for i, tkn in enumerate(word_set)}
vocab['<unk>'] = 0
vocab['<pad>'] = 1

In [9]:
embedding_layer = nn.Embedding(num_embeddings=len(vocab), 
                               embedding_dim=3,
                               padding_idx=1)

In [10]:
print(embedding_layer.weight)


Parameter containing:
tensor([[-0.8325, -0.1937, -0.2310],
        [ 0.0000,  0.0000,  0.0000],
        [-0.6540,  0.0591, -1.4180],
        [-0.0266, -0.2035,  2.4924],
        [-0.5020,  1.0316, -0.6080],
        [-0.9537, -0.9407, -2.4688],
        [ 0.2853,  1.1822, -0.1991],
        [-0.1845,  0.9914,  0.8971]], requires_grad=True)


## Bag of Words

In [11]:
!pip install konlpy

     |████████████████████████████████| 19.4 MB 1.8 MB/s 
     |████████████████████████████████| 448 kB 38.6 MB/s 


In [12]:
from konlpy.tag import Okt

okt = Okt()

def build_bag_of_words(document):
  # 온점 제거 및 형태소 분석
  document = document.replace('.', '')
  tokenized_document = okt.morphs(document)

  word_to_index = {}
  bow = []

  for word in tokenized_document:  
    if word not in word_to_index.keys():
      word_to_index[word] = len(word_to_index)  
      # BoW에 전부 기본값 1을 넣는다.
      bow.insert(len(word_to_index) - 1, 1)
    else:
      # 재등장하는 단어의 인덱스
      index = word_to_index.get(word)
      # 재등장한 단어는 해당하는 인덱스의 위치에 1을 더한다.
      bow[index] = bow[index] + 1

  return word_to_index, bow

In [13]:
doc1 = "정부가 발표하는 물가상승률과 소비자가 느끼는 물가상승률은 다르다."
vocab, bow = build_bag_of_words(doc1)
print('vocabulary :', vocab)
print('bag of words vector :', bow)

vocabulary : {'정부': 0, '가': 1, '발표': 2, '하는': 3, '물가상승률': 4, '과': 5, '소비자': 6, '느끼는': 7, '은': 8, '다르다': 9}
bag of words vector : [1, 2, 1, 1, 2, 1, 1, 1, 1, 1]


In [14]:
import re
import urllib.request
import zipfile
from lxml import etree
from nltk.tokenize import word_tokenize, sent_tokenize

## TF-IDF

TF-IDF란 출현 빈도를 사용해 어떤 단어가 문서내에서 얼마나 중요한지 나타내는 수치이다.
TF-IDF가 높을수록 그 단어가 문서를 대표하는 성질을 띈다고 할 수 있다.
- TF(term frequency): 문서 내에서 단어가 출현한 횟수
- IDF(inverse document frequency): 단어가 출현한 문서 수의 역수

TF가 높고, IDF가 적을수록 TF-IDF가 높다. 예를 들어 the는 TF가 높지만, the가 출현한 문서도 많기 때문에 그 숫자의 역인 IDF가 작아 문서 내에서 중요하지 않다고 할 수 있다.

In [15]:
import pandas as pd

In [16]:
def get_term_frequency(document, word_dict=None): # 문서 내의 단어들의 출현 빈도
    if word_dict is None:
        word_dict = {}
    words = document.split()
    
    for w in words:
        word_dict[w] = 1 + (0 if word_dict.get(w) is None else word_dict[2])
        
    return pd.Series(word_dict).sort_values(ascending=False)

In [17]:
def get_document_frequency(documents): # 각 단어가 몇 개의 문서에서 나타났는지
    dicts = []
    vocab = set([])
    df = {}
    
    for d in documents:
        tf = get_term_frequency(d)
        dicts += [tf]
        vocab = vocab | set(tf.keys())
        
    for v in list(vocab):
        df[v] = 0
        for dict_d in dicts:
            if dict_d.get(v) is not None:
                df[v] += 1
    
    return pd.Series(df).sort_values(ascending=False)
        

In [18]:
from operator import itemgetter
import numpy as np

In [19]:
def get_tfidf(docs):#TF-IDF 계산
    vocab = {}
    tfs = []
    for d in docs:
        vocab = get_term_frequency(d, vocab)
        tfs += [get_term_frequency(d)]
    df = get_document_frequency(docs)
    
    stats = []
    for word, freq in vocab.items():
        tf_idfs = []
        for idx in range(len(docs)):
            if tfs[idx].get(word) is not None:
                tf_idfs += [tfs[idx][word] * np.log(len(docs) / df[word])]
            else:
                tf_idfs += [0]
                
        stats.append((word, freq, *tf_idfs, max(tf_idfs)))
        
    return pd.DataFrame(stats, columns = ('word',
                                          'frequency',
                                          'doc1',
                                          'doc2',
                                          'doc3',
                                          'max')).sort_values('max', ascending=False)
                

In [20]:
doc1 = "나는 오늘 즐겁다."
doc2 = "너는 오늘 즐겁다."
doc3 = "즐거운 것은 오늘이 아니라 내일이다"

In [21]:
docs = [doc1, doc2, doc3]

In [22]:
get_tfidf(docs)

,word,frequency,doc1,doc2,doc3,max
2,나는,1,1.098612,0.000000,0.000000,1.098612
3,너는,1,0.000000,1.098612,0.000000,1.098612
4,즐거운,1,0.000000,0.000000,1.098612,1.098612
5,것은,1,0.000000,0.000000,1.098612,1.098612
6,오늘이,1,0.000000,0.000000,1.098612,1.098612
7,아니라,1,0.000000,0.000000,1.098612,1.098612
8,내일이다,1,0.000000,0.000000,1.098612,1.098612
0,오늘,2,0.405465,0.405465,0.000000,0.405465
1,즐겁다.,2,0.405465,0.405465,0.000000,0.405465


In [23]:
from sklearn.feature_extraction.text import TfidfVectorizer

corpus = [
    'All you need is love',
    'I love you',
    'I think I am in love with you'
]

tfidfv = TfidfVectorizer().fit(corpus)

In [24]:
tfidfv.transform(corpus).toarray()

array([[0.52004008, 0.        , 0.        , 0.52004008, 0.30714405,
        0.52004008, 0.        , 0.        , 0.30714405],
       [0.        , 0.        , 0.        , 0.        , 0.70710678,
        0.        , 0.        , 0.        , 0.70710678],
       [0.        , 0.46138073, 0.46138073, 0.        , 0.27249889,
        0.        , 0.46138073, 0.46138073, 0.27249889]])

In [25]:
tfidfv.vocabulary_

{'all': 0,
 'am': 1,
 'in': 2,
 'is': 3,
 'love': 4,
 'need': 5,
 'think': 6,
 'with': 7,
 'you': 8}

In [26]:
tfidfv_kor = TfidfVectorizer().fit(docs)

In [27]:
tfidfv_kor.transform(docs).toarray()

array([[0.        , 0.68091856, 0.        , 0.        , 0.        ,
        0.51785612, 0.        , 0.        , 0.51785612],
       [0.        , 0.        , 0.        , 0.68091856, 0.        ,
        0.51785612, 0.        , 0.        , 0.51785612],
       [0.4472136 , 0.        , 0.4472136 , 0.        , 0.4472136 ,
        0.        , 0.4472136 , 0.4472136 , 0.        ]])

In [28]:
tfidfv_kor.vocabulary_

{'것은': 0,
 '나는': 1,
 '내일이다': 2,
 '너는': 3,
 '아니라': 4,
 '오늘': 5,
 '오늘이': 6,
 '즐거운': 7,
 '즐겁다': 8}

## N-gram

In [ ]:
def custom_ngrams(text, n):
    text = text.split(' ')
    output = []
    for i in range(len(text) - n + 1):
        output.append(text[i:i+n])
    return output

In [ ]:
dynamite = 'Cause I\'m in the stars tonight, So watch me bring the fire and set the night alight'

In [ ]:
print(custom_ngrams(dynamite, 3))

[['Cause', "I'm", 'in'], ["I'm", 'in', 'the'], ['in', 'the', 'stars'], ['the', 'stars', 'tonight,'], ['stars', 'tonight,', 'So'], ['tonight,', 'So', 'watch'], ['So', 'watch', 'me'], ['watch', 'me', 'bring'], ['me', 'bring', 'the'], ['bring', 'the', 'fire'], ['the', 'fire', 'and'], ['fire', 'and', 'set'], ['and', 'set', 'the'], ['set', 'the', 'night'], ['the', 'night', 'alight']]


In [ ]:
from nltk import ngrams

In [ ]:
trigram = ngrams(dynamite.split(' '), 3)

In [ ]:
print(*trigram)

('Cause', "I'm", 'in') ("I'm", 'in', 'the') ('in', 'the', 'stars') ('the', 'stars', 'tonight,') ('stars', 'tonight,', 'So') ('tonight,', 'So', 'watch') ('So', 'watch', 'me') ('watch', 'me', 'bring') ('me', 'bring', 'the') ('bring', 'the', 'fire') ('the', 'fire', 'and') ('fire', 'and', 'set') ('and', 'set', 'the') ('set', 'the', 'night') ('the', 'night', 'alight')


## Word2Vec

http://w.elnn.kr/search/ 단어간의 유사도 측정

단어를 벡터로 바꿔서 단어간의 연산이 가능하도록 만들었다: 비슷한 위치의 단어는 비슷한 의미를 가진다

In [ ]:
from gensim.models import Word2Vec

In [ ]:
# define training data
sentences = [['this', 'is', 'the', 'first', 'sentence', 'for', 'word2vec'],
             ['this', 'is', 'the', 'second', 'sentence'],
             ['yet', 'another', 'sentence'],
             ['one', 'more', 'sentence'],
             ['and', 'the', 'final', 'sentence']]
# train model
model = Word2Vec(sentences, min_count=1) #min count: 빈도 작은 것은 학습하지 않기 위해서
# summarize the loaded model
print(model)

Word2Vec(vocab=14, size=100, alpha=0.025)


In [ ]:
model_result = model.wv.most_similar("sentence")
print(model_result)

[('first', 0.11600489914417267), ('word2vec', 0.0635729506611824), ('and', 0.01812838762998581), ('this', 0.00574251264333725), ('the', 0.0037496350705623627), ('more', -0.01629163697361946), ('another', -0.018906645476818085), ('yet', -0.026691444218158722), ('for', -0.0363539457321167), ('final', -0.061049576848745346)]


In [ ]:
 model.wv.most_similar("this")

[('one', 0.13517875969409943),
 ('for', 0.13102252781391144),
 ('another', 0.09881611168384552),
 ('and', 0.046358589082956314),
 ('word2vec', 0.016873788088560104),
 ('the', 0.009415708482265472),
 ('sentence', 0.005742482841014862),
 ('first', -0.0028019025921821594),
 ('is', -0.05448080226778984),
 ('second', -0.07377371937036514)]

In [ ]:
import gensim.downloader

In [ ]:
print(list(gensim.downloader.info()['models'].keys()))

['fasttext-wiki-news-subwords-300', 'conceptnet-numberbatch-17-06-300', 'word2vec-ruscorpora-300', 'word2vec-google-news-300', 'glove-wiki-gigaword-50', 'glove-wiki-gigaword-100', 'glove-wiki-gigaword-200', 'glove-wiki-gigaword-300', 'glove-twitter-25', 'glove-twitter-50', 'glove-twitter-100', 'glove-twitter-200', '__testing_word2vec-matrix-synopsis']
